# This is a demo for sfc project
Author: Martin Ptacek
Project variant: Predikce řešená dopřednou backprop. NN
Recurrent networks, 2/29. - Tapped delay line forward feed neural net or TDNN

In [ ]:
import numpy as np
import pandas as pd

In [7]:
# load dataset
path = 'dataset_slim.csv'
df = pd.read_csv(path)
print(df.head())

              DateTime  Consumption
0  2019-01-01 00:00:00         6352
1  2019-01-01 01:00:00         6116
2  2019-01-01 02:00:00         5873
3  2019-01-01 03:00:00         5682
4  2019-01-01 04:00:00         5557


In [ ]:
# Check for missing values in the data
df.isnull().sum()

DateTime       0
Consumption    0
dtype: int64

In [9]:
# Check for anomalies
df.describe()

,Consumption
count,46011.000000
mean,6587.616440
std,1043.654923
min,3889.000000
25%,5773.000000
50%,6552.000000
75%,7321.000000
max,9615.000000


In [ ]:
# Get numpy array and normalize it
array = df["Consumption"].to_numpy()

# Z-score normalization
std = np.std(array)
mean = np.mean(array)
normalized_array = (array - mean) / std

In [ ]:
# model

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    return x * (1 - x)

# Two layer neural network with one hidden layer
class TDNN(object):

    def __init__(self, delay_line_length, hidden_layer_size, learning_rate):
        self.output_layer_size = 1 # predict one next value from n previous values
        self.delay_line_length = delay_line_length
        self.hidden_layer_size = hidden_layer_size
        self.learning_rate = learning_rate
        
        self.initialize_weights()
        self.initialize_biases()
        
    def initialize_weights(self):
        """Perform a simple random initialization of the weights"""
        self.weights_hidden_layer = np.random.rand(self.delay_line_length, self.hidden_layer_size)
        self.weights_output_layer = np.random.rand(self.hidden_layer_size, self.output_layer_size)
    
    def initialize_biases(self):
        self.bias_hidden_layer = np.random.rand(1, self.hidden_layer_size)
        self.bias_output_layer = np.random.rand(1, self.output_layer_size)
        
    def forward(self, input_data):
        # TODO delete this
        assert len(input_data) == self.delay_line_length, "Input data must have the same length as the delay line"
        
        self.hidden_layer_output = sigmoid(np.dot(input_data, self.weights_hidden_layer) + self.bias_hidden_layer)
        self.output_layer_output = sigmoid(np.dot(self.hidden_layer_output, self.weights_output_layer) + self.bias_output_layer)
        return self.output_layer_output
    
    def backward(self, input_data, desired_data)
